## FEATURE ENGINEERING
---

In [18]:
# Import additional libraries for feature engineering

# For dataset manipulation
import pandas as pd
# For mathematical requirements (if any)
import numpy as np
# For graphical visualizations
import matplotlib.pyplot as plt
import seaborn as sns 

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load the data with parsed dates
df = pd.read_csv("superstore_parsed_dates.csv", parse_dates=["Order Date", "Ship Date"])

# Drop irrelevant columns
columns_to_drop = [
    "Row ID", "Order ID", "Customer ID", "Customer Name", 
    "Postal Code", "Product ID", "Product Name"
]

for col in columns_to_drop:
    if col in df.columns:
        df = df.drop(col, axis=1)

print("Dataset shape after cleaning:", df.shape)
df.head()

Dataset shape after cleaning: (9994, 14)


,Order Date,Ship Date,Ship Mode,Segment,Country,City,State,Region,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Bookcases,261.9600,2,0.00,41.9136
1,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Chairs,731.9400,3,0.00,219.5820
2,2016-06-12,2016-06-16,Second Class,Corporate,United States,Los Angeles,California,West,Office Supplies,Labels,14.6200,2,0.00,6.8714
3,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,Furniture,Tables,957.5775,5,0.45,-383.0310
4,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,Office Supplies,Storage,22.3680,2,0.20,2.5164


In [19]:
### 1. DATE-BASED FEATURES

In [20]:
# Extract date components from Order Date
df['Order_Year'] = df['Order Date'].dt.year
df['Order_Month'] = df['Order Date'].dt.month
df['Order_Quarter'] = df['Order Date'].dt.quarter
df['Order_Week'] = df['Order Date'].dt.isocalendar().week
df['Order_DayOfWeek'] = df['Order Date'].dt.dayofweek
df['Order_DayOfWeek_Name'] = df['Order Date'].dt.day_name()

# Extract date components from Ship Date
df['Ship_Year'] = df['Ship Date'].dt.year
df['Ship_Month'] = df['Ship Date'].dt.month
df['Ship_Quarter'] = df['Ship Date'].dt.quarter
df['Ship_Week'] = df['Ship Date'].dt.isocalendar().week
df['Ship_DayOfWeek'] = df['Ship Date'].dt.dayofweek

# Calculate shipping duration
df['Shipping_Duration'] = (df['Ship Date'] - df['Order Date']).dt.days

# Binary flags
df['Is_Weekend_Order'] = df['Order_DayOfWeek'].isin([5, 6]).astype(int)  # Saturday, Sunday

# Holiday season flag (Nov-Dec)
df['Is_Holiday_Season'] = df['Order_Month'].isin([11, 12]).astype(int)

print("Date-based features created")
print(f"Shipping duration range: {df['Shipping_Duration'].min()} to {df['Shipping_Duration'].max()} days")

Date-based features created
Shipping duration range: 0 to 7 days


In [21]:
### 2. SALES / PROFIT RATIO FEATURES

In [22]:
# Discounted price per unit
df['Price_Per_Unit'] = df['Sales'] / df['Quantity']

# Profit margin
df['Profit_Margin'] = df['Profit'] / df['Sales']

# Profit per unit
df['Profit_Per_Unit'] = df['Profit'] / df['Quantity']

# Discount flag
df['Has_Discount'] = (df['Discount'] > 0).astype(int)

print("Sales/profit-related features created!")

Sales/profit-related features created!


In [23]:
### 3. LOG-TRANSFORMED FEATURES (TO REDUCE SKEW)

In [24]:
import numpy as np

df['Log_Sales'] = np.log1p(df['Sales'])
df['Log_Profit'] = np.log1p(df['Profit'])
df['Log_Quantity'] = np.log1p(df['Quantity'])

print("Log-transformed features created!")

Log-transformed features created!


In [25]:
### 4. CATEGORICAL ENCODING

In [26]:
# Label encode some categorical variables
from sklearn.preprocessing import LabelEncoder

cat_cols = ['Ship Mode', 'Segment', 'Country', 'City', 'State', 'Region', 'Category', 'Sub-Category']
label_encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df[col + "_Encoded"] = le.fit_transform(df[col])
    label_encoders[col] = le

print("Categorical encoding applied!")

Categorical encoding applied!


In [27]:
### 5. INTERACTION FEATURES

In [28]:
# Sales per customer segment
df['Sales_Per_Quantity'] = df['Sales'] / df['Quantity']

# Interaction terms
df['Discount_x_Quantity'] = df['Discount'] * df['Quantity']
df['Discount_x_Sales'] = df['Discount'] * df['Sales']

# Region + Category grouped encoding
df['Region_Category'] = df['Region'] + "_" + df['Category']

print("Interaction features created!")

Interaction features created!


In [29]:
### 6. AGGREGATION FEATURES

In [30]:
# Average profit per category
category_profit = df.groupby('Category')['Profit'].transform('mean')
df['Category_Avg_Profit'] = category_profit

# Average sales per sub-category
subcat_sales = df.groupby('Sub-Category')['Sales'].transform('mean')
df['SubCategory_Avg_Sales'] = subcat_sales

# Region-level average discount
region_discount = df.groupby('Region')['Discount'].transform('mean')
df['Region_Avg_Discount'] = region_discount

print("Aggregation features made")

Aggregation features made


In [31]:
### 7. FINAL CHECK

In [32]:
print("Final dataset shape:", df.shape)
df.head()

Final dataset shape: (9994, 50)


,Order Date,Ship Date,Ship Mode,Segment,Country,City,State,Region,Category,Sub-Category,...,Region_Encoded,Category_Encoded,Sub-Category_Encoded,Sales_Per_Quantity,Discount_x_Quantity,Discount_x_Sales,Region_Category,Category_Avg_Profit,SubCategory_Avg_Sales,Region_Avg_Discount
0,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Bookcases,...,2,0,4,130.9800,0.00,0.000000,South_Furniture,8.699327,503.859633,0.147253
1,2016-11-08,2016-11-11,Second Class,Consumer,United States,Henderson,Kentucky,South,Furniture,Chairs,...,2,0,5,243.9800,0.00,0.000000,South_Furniture,8.699327,532.332420,0.147253
2,2016-06-12,2016-06-16,Second Class,Corporate,United States,Los Angeles,California,West,Office Supplies,Labels,...,3,1,10,7.3100,0.00,0.000000,West_Office Supplies,20.327050,34.303055,0.109335
3,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,Furniture,Tables,...,2,0,16,191.5155,2.25,430.909875,South_Furniture,8.699327,648.794771,0.147253
4,2015-10-11,2015-10-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,South,Office Supplies,Storage,...,2,1,14,11.1840,0.40,4.473600,South_Office Supplies,20.327050,264.590553,0.147253


In [33]:
# Save the enriched dataset with new engineered features

df.to_csv("superstore_feature_engineered.csv", index=False)
